In [ ]:
bres_trend_micro,bres_trend_agg = [make_trend(bres_proc,sector_var=v) for v in ['sector','sector_aggregated']]

In [ ]:
fig,ax = plt.subplots()

plot_trend(bres_trend_agg,ax=ax)

ax.legend(bbox_to_anchor=(1,1))

In [ ]:
fig,ax = plt.subplots()

plot_trend(bres_trend_micro,ax=ax)

ax.legend(bbox_to_anchor=(1,1))

In [ ]:
fig,ax = plt.subplots()

plot_bar(bres_trend_micro,ax,norm=True)

ax.legend(bbox_to_anchor=(1,1))

### b. Geographies

In [ ]:
lad_bres_shares_09,lad_bres_shares_17 = [make_lorenz(bres_proc,y=year) for year in [2009,2017]]

In [ ]:
fig,ax = plt.subplots(figsize=(5,6),sharey=True)

plot_lorenz(lad_bres_shares_17,ax)

ax.legend(bbox_to_anchor=(1,1))

In [ ]:
plot__histo_lorenz(lad_bres_shares_09,lad_bres_shares_17)

### Some maps

In [ ]:
plot_kwargs = {'scheme':'Fisher_Jenks','cmap':'viridis','edgecolor':'grey','linewidth':0,'legend':True}

In [ ]:
fig,ax = plt.subplots(figsize=(14,10),ncols=2)

year_comp(bres_proc,'sector_aggregated','journalism',ax=ax,**plot_kwargs)

plt.tight_layout()

In [ ]:
fig,ax = plt.subplots(figsize=(14,10),ncols=2)

sect_comp(bres_proc,'sector_aggregated',['other','journalism'],ax=ax,**plot_kwargs)

In [ ]:
fig,ax = plt.subplots(figsize=(21,10),ncols=3)

sect_comp(bres_proc,'sector',['publishing_newspapers','web_portals','computer_programming'],ax=ax,**plot_kwargs)

In [ ]:
fig,ax = plt.subplots(figsize=(14,10),ncols=2)

sect_comp(bres_proc,'sector',['tv_programming_broadcasting','radio_broadcasting'],ax=ax,**plot_kwargs)

In [ ]:
fig,ax = plt.subplots(figsize=(12,7),nrows=2,sharex=True,gridspec_kw={'height_ratios':[3,1]})

sectors = ['artificial_intelligence','advertising','creative_content','news_high','public_news']

for n,s in enumerate(sectors):
    
    (100*pd.crosstab(cb['year'],cb[s]>0.75,normalize=0)).loc[np.arange(2000,2019)][True].rolling(window=3).mean().dropna().plot(
        ax=ax[0],color=colors[n],linewidth=3 if 'news' in s else 1)

ax[0].set_ylabel('% of all companies')
ax[0].legend(sectors,bbox_to_anchor=(1,1))


news = cb.loc[cb['news_high']==True]
    
(100*pd.crosstab(news['year'],news['public_news']>0.75,normalize=0)).loc[np.arange(2000,2019)][True].rolling(window=3).mean().dropna().plot(
    ax=ax[1],color='blue',linewidth=3)

ax[1].set_ylabel('% of all \n news companies')


plt.tight_layout()

plt.savefig('../../reports/figures/research_slides/cb/activity_trends.pdf')

In [ ]:
100*pd.crosstab(cb['year'],cb['public_news']>0.75,normalize=0).loc[np.arange(2000,2019)][True][2018]

#### Evolution of funding?

We get the CB funding data and match it with companies

That will allow us to get levels of funding and funders for various sources

#### Analysis

Next steps:
* Create dummies for news, public interest news and AI and look at trends and actors

In [ ]:
rel_sets = [set(cb.loc[cb[s]>0.75]['id']) for s in ['artificial_intelligence','advertising','creative_content','news_high','public_news']]

In [ ]:
cb_fr_df['ai'],cb_fr_df['advertising'],cb_fr_df['creative_content'],cb_fr_df['news'],cb_fr_df['pi_news'] = [
    [x in one_set for x in cb_fr_df['company_id']] for one_set in rel_sets]

cb_fr_df['any_sector'] = 1

rel_sectors = ['ai','advertising',
               'creative_content',
               'news','pi_news','any_sector']

In [ ]:
ax = cb_fr_df.groupby('year')[rel_sectors[:-1]].sum().loc[np.arange(2000,2019)].rolling(window=3).mean().dropna().plot(color=colors)

ax.set_ylabel('Number of deals')


Totals raised

In [ ]:
fig,ax = plt.subplots(figsize=(12,7),nrows=2,sharex=True,gridspec_kw={'height_ratios':[3,1]})

total_raised = pd.concat([cb_fr_df.loc[cb_fr_df[s]==True].groupby('year')['raised_amount_usd'].sum() for s in rel_sectors],axis=1).fillna(0)/1e9

total_raised.columns = rel_sectors

total_raised.loc[np.arange(2000,2020),rel_sectors[:-1]].rolling(window=3).mean().dropna().plot(color=colors,ax=ax[0])

ax[0].set_ylabel('$ Billion')

news_funding= cb_fr_df.loc[cb_fr_df['news']==True]

(100*news_funding.groupby(['year','pi_news'])['raised_amount_usd'].sum().reset_index(drop=False).pivot(
    index='year',columns='pi_news',values='raised_amount_usd').apply(lambda x: x/x.sum(),axis=1).loc[np.arange(2000,2019)].fillna(0).rolling(
    window=3).mean()).dropna()[True].plot(color='blue',ax=ax[1],linewidth=3)

ax[1].set_ylabel('PI news as \n  % of all news')

plt.tight_layout()

plt.savefig('../../reports/figures/research_slides/cb/funding_trends.pdf')
